# Here we will implement linear regression from scratch
## The task is to compare results of linear, polynomial and interactive regressions
### In addition we will implement different validation techniques

In [37]:
import pandas as pd
import numpy as np

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [34]:
def shuffle_data_pd(df):
    #return df.reindex(np.random.permutation(df.index))
    return df.sample(frac=1).reset_index(drop=True)

    
def shuffle_data_np(dataset):
    np.random.shuffle(dataset)
    return dataset

def example():
    df = pd.DataFrame(np.random.random((6, 2)), columns=['a', 'b'])
    num = np.random.random((6, 2))
    print(df)
    print(shuffle_data_pd(df))

    print(num)

    print(shuffle_data_np(num))


In [35]:
def k_fold_cv_split_hands(dataset, k):
    N = len(dataset)
    #shuffle
    indexes = np.arange(0, N)
    indexes = shuffle_data_np(indexes)
    
    # splitting
    
    trains = []
    tests = []
    
    fraction = 1. / k
    print(fraction)
    
    for i in range(k):
        trains.append([])
        tests.append([])
        
        bottom = (int)(fraction * i * N)
        top = (int)(fraction * (i+1) * N)
        if top > N:
            top = N

        tests[i] = indexes[bottom:top]
        
        train1 = indexes[0:bottom]
        train2 = indexes[top:N]
        trains[i] = np.concatenate((train1, train2))
        
    return trains, tests
    
def k_fold_cv(dataset, k):
    pass
    
def cv_example():
    df = pd.DataFrame(np.random.random((18, 2)), columns=['a', 'b'])
    print(df)
    train_indexes, test_indexes = k_fold_cv_split_hands(df, 3)
    print(train_indexes)
    print(test_indexes)



In [97]:
def k_fold_cv(dataset, k, method,  method_type, pred_cols):
    def get_error(predicted, actual, method_type):
        if method_type is 'regression': #MSE
            summ = 0
            for i in range(len(predicted)):
                summ += (predicted[i] - actual[i]) ** 2
            return summ / len(predicted)
        else: # Classification error
            counter = 0 # WORK WITH PANDAS DF
            print(actual)
            actual = actual['species']
            print(actual)

            for i in range(len(predicted)):
                counter += predicted[i] != actual[i]
            return counter
    
    mses = []
    X = dataset[pred_cols]
    Y = dataset[[col for col in dataset.columns if col not in pred_cols]]
    
    trains, tests = k_fold_cv_split_hands(dataset, k)
    for i in range(k):
        train = trains[i]
        test = tests[i]
        train_X, train_Y = X.iloc[train, :], Y.iloc[train, :]
        test_X, test_Y = X.iloc[test, :], Y.iloc[test, :]
        method.fit(train_X, train_Y)
        predict_Y = method.predict(test_X)
        mse = get_error(predict_Y, test_Y, method_type)
        mses.append(mse)
    mmse = sum(mses) / len(mses)
    return mses, mmse

    

In [98]:
df = pd.read_csv('datasets/iris.csv')
#print(df)
regr = linear_model.LogisticRegression()
mses, mmse = k_fold_cv(df, 5, regr, 'classification', ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])

0.2
        species
51   versicolor
81   versicolor
123   virginica
102   virginica
17       setosa
84   versicolor
62   versicolor
58   versicolor
91   versicolor
141   virginica
138   virginica
65   versicolor
61   versicolor
140   virginica
11       setosa
113   virginica
71   versicolor
97   versicolor
20       setosa
14       setosa
136   virginica
35       setosa
83   versicolor
75   versicolor
5        setosa
37       setosa
24       setosa
144   virginica
125   virginica
28       setosa
51     versicolor
81     versicolor
123     virginica
102     virginica
17         setosa
84     versicolor
62     versicolor
58     versicolor
91     versicolor
141     virginica
138     virginica
65     versicolor
61     versicolor
140     virginica
11         setosa
113     virginica
71     versicolor
97     versicolor
20         setosa
14         setosa
136     virginica
35         setosa
83     versicolor
75     versicolor
5          setosa
37         setosa
24         setosa
144     virgin

C:\Users\evger\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyError: 0

In [76]:
df = pd.DataFrame(np.random.random((6, 2)), columns=list('AB'))
df.iloc[[2, 3], :]

,A,B
2,0.291993,0.575017
3,0.838193,0.430011
